In [ ]:
import os
from dotenv import load_dotenv
from typing import TypedDict, Literal
from pydantic import BaseModel, Field

load_dotenv()


True

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_deepseek import ChatDeepSeek
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate


In [ ]:
LLMS = {}

LLMS['gemini'] = ChatGoogleGenerativeAI(
    api_key = os.getenv("GOOGLE_API_KEY"),
    model = 'gemini-1.5-flash'
)

LLMS['deepseek'] = ChatDeepSeek(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    api_base=os.getenv("OPENROUTER_BASE_URL"),
    model="deepseek/deepseek-chat-v3-0324:free"
)

LLMS['moonshotai'] = ChatGroq(
    api_key = os.getenv("GROQ_API_KEY"),
    model = "moonshotai/kimi-k2-instruct"
)


In [ ]:
llm: ChatGroq = LLMS['deepseek']


In [ ]:
class ReplyState(TypedDict):
    review_text: str
    sentiment: str
    reply_text: str
    issue_type: str
    tone: str
    urgency: str


In [ ]:
class SentimentSchema(BaseModel):
    sentiment: Literal["positive", "negative"] = Field(description="Sentiment of the review text either positive or negative.")
    
def determine_sentiment(state: ReplyState):
    review_text = state['review_text']
    structered_llm1 = llm.with_structured_output(SentimentSchema)
    sentiment_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are an expert at understanding human language and emotion. Your task is to read a review and decide whether the overall sentiment is Positive or Negative.

        Focus on the review's tone, intent, and emotional impact—not just isolated words. If the review contains mixed signals, choose the side that clearly dominates. Be confident and consistent.

        Return your answer in exactly this format in lower case:
        sentiment: positive
        or
        sentiment: negative

        Don't add explanations, confidence scores, or extra text. Just the sentiment.
        """),
        ("human", "Tell me the sentiment of this review text: {review_text}")
    ])
    chain1 = sentiment_prompt | structered_llm1
    result1 = chain1.invoke(input={
        'review_text': review_text
    })
    
    return {'sentiment': result1.sentiment}


In [ ]:
class DiagnosisSchema(BaseModel):
    issue_type: str = Field(description="The type of a issue in the review text.")
    tone: str = Field(description="The tone of a user.")
    urgency: str = Field(description="The urgency level according to the review text.")
    
def run_diagnosis(state: ReplyState) -> ReplyState:
    review_text = state['review_text']
    structered_model2 = llm.with_structured_output(DiagnosisSchema);
    diagnosis_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a detailed reviewer assistant, responsible for analyzing negative reviews and providing insights on three key areas:

        Issue Type: Identify the main problem or complaint the user is expressing. This could relate to the product's functionality, quality, customer service, shipping, or any other category of dissatisfaction.

        Tone: Determine the overall emotional tone of the review. Is the tone angry, frustrated, disappointed, regretful, sad, or neutral (despite the negativity)? This reflects how strongly the reviewer feels about the issue.

        Urgency: Assess how urgent the reviewer's concern seems. Is this an issue the reviewer feels needs immediate attention, or is it something that could be resolved over time? Classify the urgency as High, Medium, or Low based on the intensity of the complaint and how urgently the reviewer asks for resolution.
        
        # Example
        Input - I ordered this laptop two weeks ago, and it still hasn't shipped. I've called customer service three times, and they just keep giving me the runaround. I need my laptop now!
        Output -
        issue_type: shipping delay
        tone: frustrated
        urgency: high
        """),
        ("human", "Diagnosis this review: {review_text}")
    ])
    chain2 = diagnosis_prompt | structered_model2
    result2 = chain2.invoke(input={
        'review_text': review_text
    })
    
    return {'issue_type': result2.issue_type, 'tone': result2.tone, 'urgency': result2.urgency}


In [ ]:
class ReplySchema(BaseModel):
    reply: str = Field(description="The reply of a review, according to the review.")


In [ ]:
def negative_response(state: ReplyState):
    review_text = state['review_text']
    structered_llm3 = llm.with_structured_output(ReplySchema);
    negative_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a customer support assistant responding to a negative review. Your task is to craft a reply based on the following parameters:
        
        - Issue Type: Identify the main problem or complaint mentioned in the review. Whether it's related to the product, service, delivery, or any other concern, address the core issue directly and empathetically.
        - Tone: Match the tone of your response to the tone of the review. If the review is angry or frustrated, your response should be apologetic and empathetic. If the tone is more neutral or regretful, your reply should be understanding but professional.
        - Urgency: If the reviewer indicates a sense of urgency (e.g., a shipping delay or a defective product), prioritize your response to assure them that their concern will be addressed quickly. If the urgency is low, your response can be more general but still respectful.

        Reply Guidelines:
        - Acknowledge the issue and empathize with the customer's experience.
        - Apologize for any inconvenience caused.
        - Provide a clear course of action, if applicable (e.g., refund, replacement, or support).
        - Offer a solution or next steps based on the issue type and urgency.
        - Maintain a friendly, professional tone throughout the response.
        
        # Output Format
        Output must only have a respond text, no explainations, no extra lines, nothing else.
        """),
        ("human", """
        Give a appropriate reply to this review: {review_text}.
        ---
        Here are the diagnosis of the above review: 
        Issue Type - {issue_type}
        Tone - {tone}
        Urgency - {urgency}
        """)
    ])
    chain3 = negative_prompt | structered_llm3
    result3 = chain3.invoke(input={
        'review_text': review_text,
        'issue_type': state['issue_type'],
        'tone': state['tone'],
        'urgency': state['urgency']
    })
    
    return {'reply_text': result3.reply}


In [ ]:
def positive_response(state: ReplyState):
    review_text = state['review_text']
    structered_llm4 = llm.with_structured_output(ReplySchema);
    positive_prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are a customer support assistant responding to a positive review. Your goal is to craft a warm, appreciative, and professional response based on the review content.
        - Tone: Reflect the tone of the review in your reply. If the review is enthusiastic, excited, or grateful, mirror that energy in your response. If the review is more neutral but positive, maintain a friendly and appreciative tone.
        - Acknowledgment: Express gratitude for the positive feedback. Let the reviewer know you appreciate their time in leaving the review and their support for your product or service.
        - Personalization: Try to personalize your reply by referring to specific elements of the review, whether it's a particular feature they liked, an experience they had, or a positive outcome they mentioned.
        - Future Engagement: Encourage future engagement, whether it's another purchase, reaching out for support, or sharing their experience with others. Offer a continued positive relationship.

        Reply Guidelines:
        - Show appreciation and gratitude.
        - Acknowledge any specific details mentioned in the review (e.g., a favorite feature, great customer service).
        - Keep the tone positive and welcoming.
        - End with an invitation for further engagement or contact.
        """),
        ("human", "Give an appropriate reply to this review: {review_text}")
    ])
    chain4 = positive_prompt | structered_llm4
    result4 = chain4.invoke(input={
        'review_text': review_text
    })
    
    return {'reply_text': result4.reply};


In [ ]:
def check_condition(state: ReplyState) -> Literal["positive_response", "run_diagnosis"]:
    if state["sentiment"] == "positive":
        return "positive_response"
    else:
        return "run_diagnosis"


In [ ]:
graph = StateGraph(ReplyState)

graph.add_node('determine_sentiment', determine_sentiment)
graph.add_node("run_diagnosis", run_diagnosis)
graph.add_node("negative_response", negative_response)
graph.add_node("positive_response", positive_response)

graph.add_edge(START, "determine_sentiment")
graph.add_conditional_edges("determine_sentiment", check_condition)
graph.add_edge("run_diagnosis", "negative_response")
graph.add_edge("negative_response", END)
graph.add_edge("positive_response", END)

workflow = graph.compile()


In [ ]:
initial_state = {
    'review_text': "I've been using this blender for a few weeks now, and it's amazing! It blends everything smoothly, even frozen fruit. The cleanup is super easy too. Best purchase I've made this year!"
}

final_state = workflow.invoke(initial_state)


In [ ]:
final_state


{'review_text': "I've been using this blender for a few weeks now, and it's amazing! It blends everything smoothly, even frozen fruit. The cleanup is super easy too. Best purchase I've made this year!",
 'sentiment': 'positive',
 'reply_text': "Thank you so much for your enthusiastic review! We're thrilled to hear that you're loving your blender and that it's performing so well, especially with frozen fruit and easy cleanup. It’s fantastic to know it’s one of your best purchases this year—that’s exactly the kind of experience we aim to provide. If you ever have any questions or need tips, don’t hesitate to reach out. Happy blending!"}

In [ ]:
initial_state = {
    'review_text': "I ordered a jacket that was supposed to fit like a medium, but it was way too small. I followed the size chart exactly. Very disappointed, and now I have to deal with the return process. Not happy!"
}

final_state = workflow.invoke(initial_state)


In [ ]:
final_state


{'review_text': 'I ordered a jacket that was supposed to fit like a medium, but it was way too small. I followed the size chart exactly. Very disappointed, and now I have to deal with the return process. Not happy!',
 'sentiment': 'negative',
 'reply_text': 'We’re so sorry to hear that the jacket didn’t fit as expected, despite following the size chart—this must be frustrating. We appreciate you bringing this to our attention, and we’d love to make it right for you. Please reach out to our customer service team at [support email/phone], and we’ll assist you with the return process and ensure you receive a refund or replacement as quickly as possible. We value your feedback and will use it to improve our sizing guidance. Thank you for your patience, and we apologize for the inconvenience.',
 'issue_type': 'incorrect sizing',
 'tone': 'disappointed',
 'urgency': 'medium'}